In [1]:
import sys
sys.path.append('/Users/bence/code/liouvillian_metro/')

import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
from scipy.linalg import expm
from scipy.sparse import csr_matrix
import pickle
from copy import deepcopy
from qiskit.quantum_info import DensityMatrix, Statevector, partial_trace, state_fidelity
import time

from oft_n_liouv_np import oft
from tools.classical import find_ideal_heisenberg, trotter_step_heisenberg_qt, ham_evol_qt, hamiltonian_matrix, HamHam

np.random.seed(667)
np.set_printoptions(precision=10, suppress=False)
#TODO: Compare rounded OFT with exact OFT later, to see how much rounding is still fine!
#TODO: State is rho not a pure state psi...

In [2]:
num_qubits = 2
num_energy_bits = 5
oft_precision = int(np.ceil(np.abs(np.log10(2**(-num_energy_bits)))))
print(oft_precision)
delta = 0.01
eps = 0.1
sigma = 5
bohr_bound = 0 #2 ** (-num_energy_bits + 1)
beta = 1
eig_index = 2
jump_site_index = 0

hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, [1, 1, 1, 1], signed=False, for_oft=True)
initial_state = hamiltonian.eigvecs[:, eig_index]

N = 2**num_energy_bits
N_labels = np.arange(N / 2, dtype=int)
N_labels_neg = np.arange(- N / 2, 0, dtype=int)
N_labels = np.concatenate((N_labels, N_labels_neg))
energy_labels = 2 * np.pi * N_labels / N 
time_labels = N_labels

# rand_jump_index = np.random.randint(0, num_qubits)ú
site_list = [qt.qeye(2) for _ in range(num_qubits)]
x_jump_ops = []
for q in range(num_qubits):
    site_list[q] = qt.sigmax()
    x_jump_ops.append(qt.tensor(site_list).full())

x_jump = x_jump_ops[jump_site_index]

2
Ideal spectrum:  [0.   0.45 0.45 0.45]
Rescaled coefficients:  [0.05625    0.05625    0.05625    0.05625    0.02739375 0.0111375 ]


### Energy decompositions of jump op A and initial states

In [4]:
# energy_projectors = [qt.ket2dm(qt.Qobj(energy_eigenstates[i])) for i in range(2**num_qubits)]

# Write jump operator in energy eigenbasis, A' = U^dag A U
x_jump_in_eigenbasis = hamiltonian.eigvecs.conj().T @ x_jump @ hamiltonian.eigvecs
print(x_jump)
print('Jump in eigenbasis:')
print(x_jump_in_eigenbasis)
    

[[0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]]
Jump in eigenbasis:
[[ 0.          +0.j  0.          +0.j  0.7071067812+0.j -0.7071067812+0.j]
 [ 0.          +0.j  0.          +0.j  0.7071067812+0.j  0.7071067812+0.j]
 [ 0.7071067812+0.j  0.7071067812+0.j  0.          +0.j  0.          +0.j]
 [-0.7071067812+0.j  0.7071067812+0.j  0.          +0.j  0.          +0.j]]


In [6]:
# Find decomposition of a vector in the energy eigenbasis
# vec = (hamiltonian.eigvecs[:, 0] + hamiltonian.eigvecs[:, 2]) / np.sqrt(2)
# rho = vec @ vec.conj().T
# vec_eigenbasis_coeffs = hamiltonian.eigvecs.conj().T @ vec  #* This for states
# print(vec_eigenbasis_coeffs)
# rho_eigenbasis = hamiltonian.eigvecs.conj().T @ rho @ hamiltonian.eigvecs
# print(rho_eigenbasis)


In [7]:
# Define your function that takes a single matrix entry as input
def transform_function(entry):
    # Example transformation: square the input
    return entry ** 2

def gaussian(entry):
    trafod_entry = np.exp(-((0.16 - entry) ** 2) * sigma ** 2)
    return trafod_entry

# Example numpy matrix
input_matrix = np.array([[1, 2, 3],
                         [4, 5, 6],
                         [7, 8, 9]])

# Vectorize the function (if it's not already vectorized)
vectorized_function = np.vectorize(transform_function)
vec_gaussian_fn = np.vectorize(gaussian)

# Apply the function to each entry in the matrix
output_matrix = vectorized_function(input_matrix)
output_matrix_gaussian = vec_gaussian_fn(input_matrix)

print(output_matrix)
print(output_matrix_gaussian)
matrix1 = np.array([[1, 2, 3],
                    [4, 5, 6],
                    [7, 8, 9]])

matrix2 = np.array([[9, 8, 7],
                    [6, 5, 4],
                    [3, 2, 1]])

# Perform element-wise multiplication
result_matrix = matrix1 * matrix2

print(result_matrix)

[[ 1  4  9]
 [16 25 36]
 [49 64 81]]
[[2.1829577951e-008 1.7430708966e-037 2.6844830678e-088]
 [7.9741094286e-161 4.5685630002e-255 0.0000000000e+000]
 [0.0000000000e+000 0.0000000000e+000 0.0000000000e+000]]
[[ 9 16 21]
 [24 25 24]
 [21 16  9]]


### Precompute all Bohr frequencies and just use them later

In [8]:
bohr_freqs = hamiltonian.spectrum[:, np.newaxis] - hamiltonian.spectrum
bohr_freqs

# THE THING

In [12]:

#The new idea is to just take the gaussian weighed bohr freq matrix and multiply entry by entry with jump in eigenbasis
# This multiplies coeffs of the matrix with the gaussian weights, and then we transform back to computational basis and done I think.
#* Scales in time almost exactly as diagonalization.

phase = np.floor(-0.44 * N / (2 * np.pi))
energy = 2 * np.pi * phase / N

print(f'Phase = {phase}')
print(f'Energy = {2 * np.pi * phase / N}')

def gaussian_weight(v, phase, N, sigma_t): #! Don't forget normalizations
    energy = 2 * np.pi * phase / N #TODO:
    return np.exp(-(energy - v) ** 2 * sigma_t ** 2)  #* insert nu vector here, maybe np.exp(np.array()) is faster than vecotrize

add_gaussian_weight = np.vectorize(gaussian_weight)
jump_op = x_jump
print(f' Jump op shape: {jump_op.shape}')
oft_precision = int(np.ceil(np.abs(np.log10(N**(-1)))))

t0 = time.time()
bohr_freqs = hamiltonian.spectrum[:, np.newaxis] - hamiltonian.spectrum
print(bohr_freqs)

print(f'Bohr freqs shape = {bohr_freqs.flatten()}')
# print('Bohr freqs:')
# print(bohr_freqs)
# t0 = time.time()
# oft_jump = np.einsum('ij, jk, jm, mn -> in', 
#                     hamiltonian.eigvecs.conj().T,
#                     add_gaussian_weight(bohr_freqs, phase, N, sigma),
#                     hamiltonian.eigvecs.conj().T,
#                     jump_op, optimize=True)

# tt = time.time()
# print(f'OFT time = {tt - t0}')
# gaussian_bohr_freqs = add_gaussian_weight(bohr_freqs, phase, N, sigma)
# tweighing = time.time()
# print('Gaussian freqs:')
# print(gaussian_bohr_freqs)
jump_in_eigenbasis = hamiltonian.eigvecs.conj().T @ jump_op @ hamiltonian.eigvecs
# print(f'Jump in eigenbasis shape = {jump_in_eigenbasis.shape}')
# print(jump_in_eigenbasis)

# weighted_jump_in_eigenbasis = gaussian_bohr_freqs * jump_in_eigenbasis
# oft_jump = hamiltonian.eigvecs @ weighted_jump_in_eigenbasis @ hamiltonian.eigvecs.conj().T

jump_nonzero_indices = np.nonzero(jump_in_eigenbasis)
nonzero_index_pairs = list(zip(*jump_nonzero_indices))

# For now we take energy diffs close to each other as the same energy diff (cuts off a ton of sum terms)
bohr_freqs_of_jump = np.round(bohr_freqs[jump_nonzero_indices], oft_precision+2)
uniqe_freqs = np.unique(bohr_freqs_of_jump, return_index=True)

t0 = time.time()
gauss_weighted_unique_freqs = add_gaussian_weight(uniqe_freqs[0], phase, N, sigma)
t1 = time.time()
gauss_weighted_unique_freqs_nonvec = np.exp(-(energy - uniqe_freqs[0]) ** 2 * sigma ** 2)
t2 = time.time()

print(f'Vectorized time = {t1 - t0}')
print(f'Non-vectorized time = {t2 - t1}')

# same_freq_indices_in_energy_list = [np.where(bohr_freqs_of_jump == freq)[0] for freq in uniqe_freqs[0]]

# print(f'From beginning till now = {time.time() - t0}')
# # print(jump_op_indices_for_energy)

# # same_freq_indices_of_jump_op = []
# # for v in range(len(same_freq_indices_in_energy_list)):
# #     same_freq_indices_of_jump_op.append([])
# #     for index in same_freq_indices_in_energy_list[v][0]:
# #         same_freq_indices_of_jump_op[v].append(nonzero_index_pairs[index])

# # print(f'Indices of the same Bohr Frequencies in the jump operator: {same_freq_indices_of_jump_op}')

# # oft_op = np.zeros((2**num_qubits, 2**num_qubits), dtype=complex)
# # for energy_index in range(len(same_freq_indices_of_jump_op)):
# #     index_tuples_for_energy = same_freq_indices_of_jump_op[energy_index]
# #     jump_op_indices_for_energy = tuple(np.array(i) for i in zip(*index_tuples_for_energy))
# #     oft_op[jump_op_indices_for_energy] = \
# #         x_jump_in_eigenbasis[jump_op_indices_for_energy]*gauss_weighted_unique_freqs[energy_index] #! and normalize
# # print('Same freq indices:')
# # print(same_freq_indices_in_energy_list)
# oft_op = np.zeros((2**num_qubits, 2**num_qubits), dtype=complex)
# tbeforeloop = time.time()
# for v in range(len(same_freq_indices_in_energy_list)):
#     indices_of_jump_op_for_freq = []
#     for index in same_freq_indices_in_energy_list[v]:
#         indices_of_jump_op_for_freq.append(nonzero_index_pairs[index])
        
#     jump_op_indices_for_energy = tuple(np.array(i) for i in zip(*indices_of_jump_op_for_freq))
#     oft_op[jump_op_indices_for_energy] = \
#     jump_in_eigenbasis[jump_op_indices_for_energy]*gauss_weighted_unique_freqs[v] #! and normalize and turn back to computational basis

# print(f'Loop time = {time.time() - tbeforeloop}')
# print(time.time() - t0)

Phase = -72.0
Energy = -0.44178646691106466
 Jump op shape: (4096, 4096)
Bohr freqs shape = [ 0.           -0.0023740517 -0.0161968342 ...  0.0183041865  0.0119783992
  0.          ]
Vectorized time = 0.477736234664917
Non-vectorized time = 0.0035789012908935547


In [45]:
bohr_freqs = hamiltonian.spectrum[:, np.newaxis] - hamiltonian.spectrum

print(bohr_freqs)

[[ 0.           -0.0301442841 -0.0659009742 -0.1125       -0.3375
  -0.3840990258 -0.4198557159 -0.45        ]
 [ 0.0301442841  0.           -0.0357566901 -0.0823557159 -0.3073557159
  -0.3539547416 -0.3897114317 -0.4198557159]
 [ 0.0659009742  0.0357566901  0.           -0.0465990258 -0.2715990258
  -0.3181980515 -0.3539547416 -0.3840990258]
 [ 0.1125        0.0823557159  0.0465990258  0.           -0.225
  -0.2715990258 -0.3073557159 -0.3375      ]
 [ 0.3375        0.3073557159  0.2715990258  0.225         0.
  -0.0465990258 -0.0823557159 -0.1125      ]
 [ 0.3840990258  0.3539547416  0.3181980515  0.2715990258  0.0465990258
   0.           -0.0357566901 -0.0659009742]
 [ 0.4198557159  0.3897114317  0.3539547416  0.3073557159  0.0823557159
   0.0357566901  0.           -0.0301442841]
 [ 0.45          0.4198557159  0.3840990258  0.3375        0.1125
   0.0659009742  0.0301442841  0.          ]]


### Find all possible energy jumps for a give state and jump operator A

Jump operator in the eigenbasis shows at each entry which eigenstate jumps are possible, so for each nonzero entry we can take their indices and find the possible energy differences. But of course also depending on the initial state some of these will or will not happen.

In [46]:
# Indices of nonzero entries in jump operator
jump_nonzero_indices = np.nonzero(x_jump_in_eigenbasis)
print(jump_nonzero_indices)
nonzero_index_pairs = list(zip(*jump_nonzero_indices))
print(nonzero_index_pairs)
# Convert list of tuples back to tuple of arrays
jump_nonzero_indices = tuple(np.array(i) for i in zip(*nonzero_index_pairs))
print(jump_nonzero_indices)

(array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5,
       5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7]), array([0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5,
       6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 5, 6, 0, 1, 2, 3, 4, 5,
       6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 5, 6]))
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 5), (4, 6), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 5), (7, 6)]
(array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4

In [47]:
# Get all Bohr frequencies for the nonzero jump op indices
# bohr_freqs_for_jump = bohr_freqs[jump_nonzero_indices]
bohr_freqs_for_jump = np.round(bohr_freqs[jump_nonzero_indices], oft_precision+2)
print('These are all the energy jumps in the system:')
print(bohr_freqs_for_jump)

# Bohr frequencies that are unique but up to some precision:
# uniqe_freqs = np.unique(bohr_freqs_for_jump, return_index=True)
uniqe_freqs = np.unique(np.round(bohr_freqs_for_jump, oft_precision+2), return_index=True)  #! Change this rounding if needed
print(f'Number of unique frequencies: {len(uniqe_freqs[0])}')
# Get the indices of the same Bohr frequencies
same_freq_indices_in_energy_list = [np.nonzero(bohr_freqs_for_jump == bohr_freqs_for_jump[uniqe_freqs[1][i]]) 
                                    for i in range(len(uniqe_freqs[0]))]

print(f'Unique Bohr Frequencies: {uniqe_freqs[0]}')
print(f'Indices of the same Bohr Frequencies in the list of energy diffs: {same_freq_indices_in_energy_list}')
same_freq_indices_of_jump_op = []  #* This is what I will need in the main computation multiplied by the states coeffs (in eigenbasis) of the latter index
for v in range(len(same_freq_indices_in_energy_list)):
    same_freq_indices_of_jump_op.append([])
    for index in same_freq_indices_in_energy_list[v][0]:
        same_freq_indices_of_jump_op[v].append(nonzero_index_pairs[index])
        
print(f'Indices of the same Bohr Frequencies in the jump operator: {same_freq_indices_of_jump_op}')

These are all the energy jumps in the system:
[ 0.     -0.0301 -0.0659 -0.1125 -0.3375 -0.3841 -0.4199 -0.45    0.0301
  0.     -0.0358 -0.0824 -0.3074 -0.354  -0.3897 -0.4199  0.0659  0.0358
  0.     -0.0466 -0.2716 -0.3182 -0.354  -0.3841  0.1125  0.0824  0.0466
  0.     -0.225  -0.2716 -0.3074 -0.3375  0.3375  0.3074  0.2716  0.225
 -0.0466 -0.0824  0.3841  0.354   0.3182  0.2716  0.0466  0.     -0.0358
 -0.0659  0.4199  0.3897  0.354   0.3074  0.0824  0.0358  0.     -0.0301
  0.45    0.4199  0.3841  0.3375  0.0659  0.0301]
Number of unique frequencies: 33
Unique Bohr Frequencies: [-0.45   -0.4199 -0.3897 -0.3841 -0.354  -0.3375 -0.3182 -0.3074 -0.2716
 -0.225  -0.1125 -0.0824 -0.0659 -0.0466 -0.0358 -0.0301  0.      0.0301
  0.0358  0.0466  0.0659  0.0824  0.1125  0.225   0.2716  0.3074  0.3182
  0.3375  0.354   0.3841  0.3897  0.4199  0.45  ]
Indices of the same Bohr Frequencies in the list of energy diffs: [(array([7]),), (array([ 6, 15]),), (array([14]),), (array([ 5, 23]),), (a

In [48]:
# Construct jump operators for each Bohr frequency (not necessary for main computation but good for checking)

#! Dude when you do this part just multiply with gaussian factor at the right nű and it's done, that's A(w) if it's summed up

print(x_jump_in_eigenbasis)
jump_ops_for_energy = []
for energy_index in range(len(same_freq_indices_of_jump_op)):
    index_tuples_for_energy = same_freq_indices_of_jump_op[energy_index]
    jump_op_indices_for_energy = tuple(np.array(i) for i in zip(*index_tuples_for_energy))
    jump_op_for_energy = np.zeros((2**num_qubits, 2**num_qubits), dtype=complex)
    jump_op_for_energy[jump_op_indices_for_energy] = x_jump_in_eigenbasis[jump_op_indices_for_energy]
    jump_ops_for_energy.append(jump_op_for_energy)

for v in range(len(jump_ops_for_energy)):
    print(f'Jump operator for energy: {np.round(uniqe_freqs[0][v], 3)}')
    print(jump_ops_for_energy[v])



[[-6.0109845572e-16+0.j -6.2796303020e-01+0.j  4.6905520958e-16+0.j
  -2.3551386880e-16+0.j  7.0710678119e-01+0.j -1.5027461393e-16+0.j
   3.2505758367e-01+0.j -2.0299618927e-16+0.j]
 [-6.2796303020e-01+0.j  1.0481036776e-15+0.j  6.7388733868e-01+0.j
  -1.7077328455e-16+0.j -2.2204460493e-16+0.j -2.1418649530e-01+0.j
  -1.2819751243e-16+0.j -3.2505758367e-01+0.j]
 [ 4.6905520958e-16+0.j  6.7388733868e-01+0.j -5.1980425690e-16+0.j
  -5.0000000000e-01+0.j  5.0000000000e-01+0.j -1.9626155734e-17+0.j
   2.1418649530e-01+0.j -1.6258839764e-17+0.j]
 [-2.3551386880e-16+0.j -1.7077328455e-16+0.j -5.0000000000e-01+0.j
   1.5700924587e-16+0.j -1.1102230246e-16+0.j -5.0000000000e-01+0.j
  -8.8398756872e-17+0.j -7.0710678119e-01+0.j]
 [ 7.0710678119e-01+0.j -2.2204460493e-16+0.j  5.0000000000e-01+0.j
  -1.1102230246e-16+0.j  0.0000000000e+00+0.j -5.0000000000e-01+0.j
  -2.7755575616e-17+0.j  0.0000000000e+00+0.j]
 [-1.5027461393e-16+0.j -2.1418649530e-01+0.j -1.9626155734e-17+0.j
  -5.0000000000e-

In [49]:
# Check if sum gives the whole
# np.allclose(x_jump_in_eigenbasis, np.sum(jump_ops_for_energy, axis=0), atol=1e-12)

In [50]:
# Generate random sparse matrix 
# qubits = 13
# matrix = np.zeros((2**qubits, 2**qubits), dtype=complex)
# matrix[1, 2] = 1
# matrix[2, 1] = 1
# matrix[3, 3] = 1
# matrix[4, 4] = 1
# matrix[5, 10] = 1
# matrix[10, 5] = 1


# #turn to scipy sparse
# sp_matrix = csr_matrix(matrix)

# #random dense numpy matrix
# matrix2 = np.random.rand(2**qubits, 2**qubits) + 1j * np.random.rand(2**qubits, 2**qubits)

# t0 = time.time()
# # multiply the two dense matrices
# np.dot(matrix, matrix2)
# t1 = time.time()
# print('Dense matrix multiplication time: ', t1 - t0)

# sp_matrix2 = csr_matrix(matrix2)
# t0 = time.time()
# # multiply the dense with the sparse matrices
# np.dot(sp_matrix, sp_matrix2)
# t1 = time.time()
# print('Sparse matrix multiplication time: ', t1 - t0)



In [869]:

#! I don't think we should compute this just to truncate the grid, because at that point we basically computed the whole thing.
#! Let's see how much gets truncated by just using the Gaussian cut off

def get_truncated_energy_axis(jump_op: np.ndarray, initial_dm: np.ndarray, hamiltonian: HamHam, 
                              N: int, sigma: float) -> np.ndarray:
    
    phase_labels = np.arange(N / 2, dtype=int)
    phase_labels_neg = np.arange(- N / 2, 0, dtype=int)
    phase_labels = np.concatenate((phase_labels, phase_labels_neg))
    energy_labels = 2 * np.pi * phase_labels / N
    
    print(truncated_phase_indices)
    print(energy_labels[truncated_phase_indices])
    print(f'Ratio of truncated to full grid = {len(truncated_phase_indices) / N}')
    
    
    # oft_precision = int(np.ceil(np.abs(np.log10(N**(-1)))))

    # bohr_freqs = hamiltonian.spectrum[:, np.newaxis] - hamiltonian.spectrum
    # # print('All Bohr frequencies')
    # # print(bohr_freqs)
    # # initial_dm_eigen = hamiltonian.eigvecs.conj().T @ initial_dm @ hamiltonian.eigvecs
    # # jump_op_eigen = hamiltonian.eigvecs.conj().T @ jump_op @ hamiltonian.eigvecs
    
    # # jump_nonzero_indices = np.nonzero(jump_op_eigen)
    # bohr_freqs_of_jump = np.round(bohr_freqs[jump_nonzero_indices], oft_precision+3)
    # # print(f'Bohr freqs of jump = {bohr_freqs_of_jump}')
    # uniqe_freqs = np.unique(bohr_freqs_of_jump) #return_index=True)
    # unique_closest_energy_labels = (uniqe_freqs * N / (2 * np.pi))[:, np.newaxis]
    # print(unique_closest_energy_labels)
    
    
    # # Find all jump operator indices that correspond to the same Bohr frequency
    # # same_freq_indices_in_energy_list = [np.nonzero(bohr_freqs_of_jump == bohr_freqs_of_jump[uniqe_freqs[1][i]]) 
    # #                                 for i in range(len(uniqe_freqs[0]))]
    # # same_freq_indices_of_jump_op = []
    # # for v in range(len(same_freq_indices_in_energy_list)):
    # #     same_freq_indices_of_jump_op.append([])
    # #     for index in same_freq_indices_in_energy_list[v][0]:
    # #         same_freq_indices_of_jump_op[v].append(nonzero_index_pairs[index])
    # # print(f'Jump op indices for the unique energies: {uniqe_freqs[0]}')
    # # print(same_freq_indices_of_jump_op)
    
    
     
    # # distance from 0 of gaussian that has amplitudes above epsilon 
    # ft_gauss = lambda energy: np.exp(-sigma**2 * energy**2)
    # ft_gauss_amplitudes = np.array([ft_gauss(energy) for energy in energy_labels])  # Binary ordered
    # ft_gauss_normalization = np.linalg.norm(ft_gauss_amplitudes)
    # eps = 1e-4
    # cut_off_dist = np.sqrt(-np.log(ft_gauss_normalization * eps) / sigma**2)  # in energy units
    # cut_off_num_labels = int(np.ceil(cut_off_dist * N / (2 * np.pi)))  # to left and right
    
    # # Truncated labels around unique energy labels
    # truncated_regions = np.concatenate((unique_closest_energy_labels - cut_off_num_labels, 
    #                                     unique_closest_energy_labels + cut_off_num_labels), axis=1)

random_initial_dm = np.random.rand(2**num_qubits, 2**num_qubits) + 1j * np.random.rand(2**num_qubits, 2**num_qubits)
truncated_grid = get_truncated_energy_axis(x_jump, random_initial_dm, hamiltonian, N, sigma)

[   0    1    2    3    4    5    6    7    8    9   10   11   12   13
   14   15   16   17   18   19   20   21   22   23   24   25   26   27
   28   29   30   31   32   33   34   35   36   37   38   39   40   41
   42   43   44   45   46   47   48   49   50   51   52   53   54   55
   56   57   58   59   60   61   62   63   64   65   66   67   68   69
   70   71   72   73  951  952  953  954  955  956  957  958  959  960
  961  962  963  964  965  966  967  968  969  970  971  972  973  974
  975  976  977  978  979  980  981  982  983  984  985  986  987  988
  989  990  991  992  993  994  995  996  997  998  999 1000 1001 1002
 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016
 1017 1018 1019 1020 1021 1022 1023]
[ 0.            0.0061359232  0.0122718463  0.0184077695  0.0245436926
  0.0306796158  0.0368155389  0.0429514621  0.0490873852  0.0552233084
  0.0613592315  0.0674951547  0.0736310778  0.079767001   0.0859029241
  0.0920388473  0.0981747704  0.10431069

In [868]:
randstate_better = np.zeros(2**(num_qubits))
randstate_better[np.random.choice(2**num_qubits, 2**num_qubits//2, replace=False)] = 1
randstate_better /= np.linalg.norm(randstate_better)

energy_of_random_state = np.real(randstate_better.conj().T @ hamiltonian.qt.full() @ randstate_better)
print(f'Energy of random state = {energy_of_random_state}')

Energy of random state = 0.309501876658944
